# Bridge Inspection Data Manipulation

*Note: This is the first stage of an ongoing project at the University of Waterloo.*

### <font color='blue'>*Author: Ali Chehrazi*</font>




## 1. Description
Bridges will deteriorate over time due to corrosion, cracks, etc. Global warming can affect the rate of deterioration of bridge structures. The aim of the project is to evaluate the effect of global warming on the deterioration of bridge structures.

The goal of this notebook, the first stage of the project, is to collect a clean bridge inspection dataset from a big bridge inspection dataset provided by the National Bridge Inventory (NBI) of the United States.

During the inspection, the condition state of a bridge is recorded. The condition state of a bridge is a number between 0 and 9 (0 is a failure and 9 is excellent condition). These condition states can be used to determine the probability of moving from one condition state to the next condition over a given time period.

<img src="Image/Brooklyn-Bridge.jpg">


### Bridge Selection Criteria
Apart from problems like missing information in the dataset, there are a few criteria for choosing the bridges for the clean dataset:

1. The condition state of a bridge should not increase over time. Basically, the bridge condition should not get better over time. Typically, a bridge can get some repair, or "rehabilitation", over time and its condition can get better by repairs. However, here we are not interested in those bridges. Only the bridges that haven't received any repair and naturally went from one condition to the next condition provide valuable information for us.

2. At least the condition state data for 15 years should be available for the selected bridges.

3. Only one level of deterioration is allowed each year.

Examples:

If the condition state (CS) of a bridge is 5 in 2008, it cannot be any number higher than 5 in 2009. (Because of Criterion 1)

If the condition state (CS) of a bridge is 9 in 2008, it can only be 9 or 8 in 2009. (Because of Criterion 3)

### About data
The database includes the information for the bridges in the state of New York from 1992 to 2023 (one CSV file for each year). The recorded information is very comprehensive including 134 items for each bridge over the years. We are only interested in the condition state (CS). However, we will get some other data about the location of the bridge, the built year, etc. The exact parameters we are going to get from the main database are as follows:

STRUCTURE_NUMBER_008': A unique number for each bridge

'LAT_016': Lattitude of the location of the bridge

'LONG_017': Longitude of the location of the bridge

'YEAR_BUILT_027': The year the bridge was built.

'DECK_COND_058': The condition state of the bridge deck (0 to 9).

'DECK_STRUCTURE_TYPE_107': The material of the bridge deck (1 and 2 for concrete, our interest is concrete material)

Further information about all the recorded data is available in the pdf file: https://www.fhwa.dot.gov/bridge/mtguide.pdf

The data can be found in the following link: https://www.fhwa.dot.gov/bridge/nbi/ascii.cfm

## 2. Importing libraries
In this notebook, we are going to use Pandas DataFrame to prepare a clean database.

In [1]:
import pandas as pd # Pandas dataframes will be used frequently in this notebook.
import folium  # this library will be used to show the location of the bridge over the map.

## 3. Merging CSV files with the required feature
The information for each year is provided in a separate CSV file. Also, for each bridge, 134 item is reported. In this part, let's get the items that we need and merge the CSV files in one DataFrame.

Let's look at the CSV file for the year 2000.

In [2]:
df=pd.read_csv("/content/drive/MyDrive/Ny_Condtion_States/Ny2000.txt",low_memory=False)

In [3]:
df

,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,PROJ_NO,PROJ_SUFFIX,NBI_TYPE_OF_IMP,DTL_TYPE_OF_IMP,SPECIAL_CODE,STEP_CODE,STATUS_WITH_10YR_RULE,SUFFICIENCY_ASTERC,SUFFICIENCY_RATING,STATUS_NO_10YR_RULE
0,36,000000001000040,1,2.0,1.0,00001,0.0,87,119.0,44831.0,...,NaN,0.0,NaN,NaN,NaN,NaN,2,NaN,37.9,2
1,36,000000001000090,1,2.0,1.0,00001,0.0,87,119.0,64309.0,...,NaN,0.0,NaN,NaN,NaN,NaN,2,NaN,68.5,2
2,36,000000001000121,1,2.0,1.0,00001,0.0,87,119.0,59223.0,...,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,87.8,0
3,36,000000001000122,1,2.0,1.0,00001,0.0,87,119.0,59223.0,...,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,74.7,0
4,36,000000001000140,1,3.0,1.0,00032,0.0,11,1.0,46536.0,...,7493300,0.0,12.0,15.0,NaN,6,1,NaN,48.7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22116,36,000000007715040,2,4.0,0.0,00000,0.0,NaN,55.0,72455.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22117,36,000000007715150,2,1.0,1.0,00590,0.0,NaN,55.0,8246.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22118,36,000000007715160,2,1.0,1.0,00590,0.0,NaN,55.0,8246.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22119,36,000000007715240,2,1.0,4.0,00890,0.0,NaN,93.0,29366.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's take a look at the paramters of intrest.

In [4]:
df=df[['STRUCTURE_NUMBER_008','LAT_016','LONG_017','YEAR_BUILT_027','DECK_COND_058','DECK_STRUCTURE_TYPE_107']].copy()
df

,STRUCTURE_NUMBER_008,LAT_016,LONG_017,YEAR_BUILT_027,DECK_COND_058,DECK_STRUCTURE_TYPE_107
0,000000001000040,40565400.0,73440000.0,1895.0,N,N
1,000000001000090,40593000.0,73404200.0,1959.0,6,1
2,000000001000121,41004200.0,73392400.0,1926.0,6,1
3,000000001000122,41004200.0,73392400.0,1888.0,5,N
4,000000001000140,42411800.0,73434200.0,1938.0,4,1
...,...,...,...,...,...,...
22116,000000007715040,43124200.0,77580600.0,1983.0,NaN,NaN
22117,000000007715150,43083600.0,77324200.0,1989.0,NaN,NaN
22118,000000007715160,43083600.0,77323600.0,1989.0,NaN,NaN
22119,000000007715240,42510600.0,74003600.0,1998.0,NaN,NaN


Let's see the amount of missing information.


In [5]:
df.isna().sum()

STRUCTURE_NUMBER_008          0
LAT_016                      42
LONG_017                     42
YEAR_BUILT_027               42
DECK_COND_058              4734
DECK_STRUCTURE_TYPE_107    4735
dtype: int64

Our parameter of interest is DECK_COND_058. Only integers between 0 and 9 are allowed for this parameter in the clean database. Let's see if there are other inappropriate values in this column.

In [6]:
df["DECK_COND_058"].value_counts()

6    3249
7    2819
5    2776
N    2707
8    2576
9    1705
4    1279
3     239
2      26
1       9
0       2
Name: DECK_COND_058, dtype: int64

It seems that there are also values of N in this column. We should get rid of the rows with Nan and N values. We only looked at the year 2000. There may be another type of inappropriate value in other years. So, let's first make a DataFrame that includes all the data from all years. Also, let's add a column "Year" to know the year for each recording in the dataset for all years. Also, let's only get the data for bridges with concrete material.

In [7]:
for i in range(1992,2024):
  path="/content/drive/MyDrive/Ny_Condtion_States/Ny"+str(i)+".txt"       # this will make a string with the path for the csv file for each year
  df=pd.read_csv(path, low_memory=False)      # this will read the csv file for each year
  df=df[['STRUCTURE_NUMBER_008','LAT_016','LONG_017','YEAR_BUILT_027','DECK_COND_058','DECK_STRUCTURE_TYPE_107']].copy() # let's get the information that we need from the dataset
  df.insert(1,'Year',i)     # this will insert a column called year that basically has the year for each recording.

  df=df.loc[(df['DECK_STRUCTURE_TYPE_107']=='1') | (df['DECK_STRUCTURE_TYPE_107']=='2')]  # 1 and 2 is for concrete material which we are intrested in.

  if i==1992:   #this if statement is used to merge the data in a data frame called df_all_years
    df_all_years=df
  else:
    df_all_years=pd.concat([df_all_years, df], ignore_index=True)

We constructed a DataFrame including the data from all years. Now, let's take a look at the DataFrame.

In [8]:
df_all_years

,STRUCTURE_NUMBER_008,Year,LAT_016,LONG_017,YEAR_BUILT_027,DECK_COND_058,DECK_STRUCTURE_TYPE_107
0,000000001000090,1992,40593000.0,73404200.0,1959.0,6,1
1,000000001000121,1992,41004200.0,73392400.0,1926.0,5,1
2,000000001000140,1992,42411800.0,73434200.0,1938.0,4,1
3,000000001000160,1992,42421800.0,73414800.0,1932.0,4,1
4,000000001000190,1992,42440000.0,73375400.0,1931.0,5,1
...,...,...,...,...,...,...,...
304361,DAPFORDRU000061,2023,44073137.0,75382396.0,2004.0,8,1
304362,DAPUSMWPT000006,2023,41240997.0,73581348.0,1970.0,7,1
304363,DAPUSMWPT000009,2023,41234643.0,73573222.0,1940.0,4,1
304364,DAPUSMWPT000015,2023,41232882.0,73571652.0,1950.0,7,1


Now, let's see the values in "DECK_COND_058" and get rid of the inappropriate values.

In [9]:
df_all_years["DECK_COND_058"].value_counts()

6    66384
5    61698
7    57140
8    46893
4    40847
9    21866
N     4916
3     4390
2      134
1       58
0       11
Name: DECK_COND_058, dtype: int64

So, let's get rid of rows with N or Nan values in the column "DECK_COND_058".

In [10]:
df_all_years=df_all_years[~((df_all_years["DECK_COND_058"].isna()) | (df_all_years["DECK_COND_058"]=='N'))]

In [11]:
df_all_years.isna().sum()

STRUCTURE_NUMBER_008       0
Year                       0
LAT_016                    6
LONG_017                   6
YEAR_BUILT_027             1
DECK_COND_058              0
DECK_STRUCTURE_TYPE_107    0
dtype: int64

Now, all the rows in the DataFrame include bridge condition records. Let's see the data type of each column.

In [12]:
df_all_years.dtypes

STRUCTURE_NUMBER_008        object
Year                         int64
LAT_016                    float64
LONG_017                   float64
YEAR_BUILT_027             float64
DECK_COND_058               object
DECK_STRUCTURE_TYPE_107     object
dtype: object

Let's correct the data type for each column.

In [13]:
df_all_years = df_all_years.astype({'DECK_COND_058':'int', 'DECK_STRUCTURE_TYPE_107':'int'})
df_all_years.dtypes

STRUCTURE_NUMBER_008        object
Year                         int64
LAT_016                    float64
LONG_017                   float64
YEAR_BUILT_027             float64
DECK_COND_058                int64
DECK_STRUCTURE_TYPE_107      int64
dtype: object

Now, Let's correct the database based on the required selection criteria. Reminder, the selection criteria are:

1. The condition state of a bridge should not increase over time. Basically, the bridge condition should not get better over time. Typically, a bridge can get some repair, or "rehabilitation", over time and its condition can get better by repairs. However, here we are not interested in those bridges. Only the bridges that haven't received any repair and naturally went from one condition to the next condition provide valuable information for us.

2. At least the condition state data for 15 years should be available for the selected bridges.

3. Only one level of deterioration is allowed each year.


    


Let's save the clean data in a new DataFrame. Also, let's save the name of the bridges with a clean recording (meeting all 3 criteria), their location, and the number of records for each bridge in a nested list.

In [ ]:
df_all_years.sort_values(by=['STRUCTURE_NUMBER_008','Year'], inplace=True) #Let's sort the results by structure number then year of the recording.
t=1 # this will be used to see if Criteria 2 and 3 are OK. t==1 when bridge recordings meet both criteria.
j=0 # this will be used to count the number of readings (years) for each bridge. Bridges with lower than 15 years of record will be excluded.
i=0 # this will be used to go over the DataFrame one by one.

Clean_readings=[] # the bridges that meet the criteria and the number of records for each bridge will be saved in this nested lits.
df_clean=df_all_years[0:0]  #this dataframe saves the recordings that meet the required criteria

n=len(df_all_years) # length of the data frame

str_numi=df_all_years.iloc[0,0] # the structure number of the first bridge in the dataframe
CS=df_all_years.iloc[0,5]  # the condition state of the the first bridge in the dataframe in the first recorded year



while i<n:   # the while loop goes over all the recording in the dataframe
  if df_all_years.iloc[i,0]==str_numi:  # this if statement checks if the recordings for a specific bridge has ended (by checking the structure number whihc is unique for each bridge.)
    if (df_all_years.iloc[i,5]>CS):      # this if statement checks if the second criteria is not true.
      t=0
    if (CS-df_all_years.iloc[i,5]>1):    # this if statement checks if the third criteria is not true.
      t=0
    CS=df_all_years.iloc[i,5]
    j+=1                        # this adds 1 to the number of recordings for a specific bridge
  else:                         # if the structure number of the bridge changes, the else statment save the results for the previous bridge only if it meets all the required criteria.
    if (t==1) & (j>15):         # this if statement checks if the bridge recordings meet all the criteria
      Clean_readings.append([str_numi,j,df_all_years.iloc[i-1,2],df_all_years.iloc[i-1,3]])         # the bridge structure number and the number of recording, and the location of the bridge will be saved in a nested loop.
      df_clean=df_clean.append(df_all_years[i-j:i], ignore_index = True)     # the clean recordings will be added to a dataframe.
    t=1                         #reset parameters
    CS=df_all_years.iloc[i,5]            #reset parameters
    str_numi=df_all_years.iloc[i,0]     #reset parameters
    j=1                         #reset parameters
  i+=1

In [15]:
df_clean

,STRUCTURE_NUMBER_008,Year,LAT_016,LONG_017,YEAR_BUILT_027,DECK_COND_058,DECK_STRUCTURE_TYPE_107
0,000000001000460,1992,43483000.0,76120000.0,1931.0,4,1
1,000000001000460,1993,43483000.0,76120000.0,1931.0,4,1
2,000000001000460,1994,43483000.0,76120000.0,1931.0,4,1
3,000000001000460,1995,43483000.0,76120000.0,1931.0,4,1
4,000000001000460,1996,43483000.0,76120000.0,1931.0,4,1
...,...,...,...,...,...,...,...
18633,DAPUSMWPT000036,2019,41232378.0,73571933.0,1997.0,6,1
18634,DAPUSMWPT000036,2020,41232378.0,-735719.0,1997.0,6,1
18635,DAPUSMWPT000036,2021,41232378.0,73571933.0,1997.0,6,1
18636,DAPUSMWPT000036,2022,41232378.0,73571933.0,1997.0,6,1


Everything looks nice, let's check if there is any missing information.

In [16]:
df_clean.isna().sum()

STRUCTURE_NUMBER_008       0
Year                       0
LAT_016                    0
LONG_017                   0
YEAR_BUILT_027             0
DECK_COND_058              0
DECK_STRUCTURE_TYPE_107    0
dtype: int64

Let's save this data frame to a CSV file. This will be used for the next stage of the project.



In [17]:
df_clean.to_csv('Clean_dataset.csv')

Let's see the number of the bridge in the clean dataset.

In [18]:
print("The number of bridges in the clean dataset is "+str(len(Clean_readings))+".")

The number of bridges in the clean dataset is 709.


We already saved the bridge condition recordings that pass the criteria. That dataset included condition recordings over the years. Let's make a DataFrame with general information for the bridges in that dataset.

In [19]:
Bridge_info=pd.DataFrame(Clean_readings,columns=["Structure Number","Number of recording","Latitude","Longitude"])
Bridge_info.to_csv('Bridge_info.csv')
Bridge_info

,Structure Number,Number of recording,Latitude,Longitude
0,000000001000460,32,43483065.0,76115736.0
1,000000001000540,32,44001303.0,75482265.0
2,000000001000590,32,44014739.0,75403611.0
3,000000001000640,32,44085671.0,75190374.0
4,000000001000660,16,44105332.0,75165195.0
...,...,...,...,...
704,DAPFORDRU000007,19,44113862.0,75291072.0
705,DAPFORDRU000061,18,44073137.0,75382396.0
706,DAPUSMWPT000006,28,41240997.0,73581348.0
707,DAPUSMWPT000009,28,41234643.0,73573222.0


Let's take a look at the selected bridges over the map. First, let's make a Folium map.

In [20]:
New_York_Cordinates=[43.2994, -73.935242]
Map = folium.Map(location=New_York_Cordinates, zoom_start=5)

Let's define a function to convert GPS reading to decimals. Then let's draw a circle for each selected bridge.

In [21]:
#let's define a function to convert the GPS data to decimal format for the folium map.

def GPS_Dec(input):
  return (float(str(input)[0:2])+float(str(input)[2:4])/60.0+float(str(input)[4:8])/360000.0)

for i in range(len(Bridge_info)):
    coordinate=[GPS_Dec(Bridge_info.iloc[i,2]),-GPS_Dec(Bridge_info.iloc[i,3])]
    circle=folium.Circle(coordinate, radius=500, color='#d35400', fill=True)
    Map.add_child(circle)
Map


<img src="Image/Folium map.jpg">

Please note that the Folium map may not be shown on some browsers due to security issues. Therefore, I put a screenshot from my Google Colab Notebook above.
## Summary

In this notebook, We worked with 32 CSV files that reported the bridge inspection data for the state of New York from 1992 to 2023. The main goal was to select the inspection data over the years that meet the required criteria/standards. The outputs were one CSV file with the inspection data for the selected bridges over the years called Clean_database.csv and another CSV file with the name and location of the selected bridges called Bridge_info.csv. This data will be used in the next phase of the project to determine the probability that a bridge deteriorates and goes from a better condition to a worse condition.
